In [2]:
import pandas as pd
import re
issue_db = pd.read_csv('issueDB.csv')

In [12]:
def clean_text(issue_content):
    if pd.isna(issue_content): return 'None'
    issue_content = re.sub(r'\n', ' ', issue_content)
    issue_content = re.sub(r'\t', ' ', issue_content)
    issue_content = re.sub(r'\r', ' ', issue_content)
    # Remove code block ``` xxx ```
    issue_content = re.sub(r'```.*?```', ' ', issue_content, flags=re.DOTALL)
    # Remove hyperlink [xxx](xxx)
    issue_content = re.sub(r'\[.*?\]\(.*?\)', ' ', issue_content)
    # Remove hyperlink ![xxx](xxx)
    issue_content = re.sub(r'\!\[.*?\]\(.*?\)', ' ', issue_content)
    # Remove URL http://xxx
    issue_content = re.sub(r'http\S+', ' ', issue_content)
    # Remove XML tag <xxx>
    issue_content = re.sub(r'<.*?>', ' ', issue_content)
    if re.sub(r' ', '', issue_content) == "": return 'None'
    return issue_content

In [18]:
up = 20000 # Contain
limit = 20000 # Not contain
issue_title = issue_db['Title'][up:].copy()
issue_body = issue_db['Body'][up:].copy()
assert len(issue_title) == len(issue_body)
for i in range(up, up + len(issue_title)):
    issue_title[i] = clean_text(issue_title[i])
    issue_body[i] = clean_text(issue_body[i])
print(len(issue_body))
issue_title.to_csv(f'/Users/home/Documents/PyCharm/Senti4SD/ClassificationTask/title{up}-.csv', header=False, index=False)
issue_body.to_csv(f'/Users/home/Documents/PyCharm/Senti4SD/ClassificationTask/body{up}-.csv', header=False, index=False)
test = pd.read_csv(f'/Users/home/Documents/PyCharm/Senti4SD/ClassificationTask/body{up}-.csv', header=None)
print(len(test))

4953
4953


In [24]:
prediction = pd.read_csv('/Users/home/Desktop/sentiment/body/predictions20000-.csv')
print(len(prediction))
#delete the 1st letter of the 'Row' column and convert it to int
prediction['Row'] = prediction['Row'].str[1:].astype(int)
#sort the dataframe by the 'Row' column
prediction = prediction.sort_values(by=['Row'])
print(prediction)
#output the dataframe to a csv file
prediction.to_csv('/Users/home/Desktop/sentiment/body/5.csv', index=False, header=False)

4953
       Row Predicted
7        0  positive
2        1   neutral
15       2  negative
0        3   neutral
3        4   neutral
...    ...       ...
4948  4948   neutral
4949  4949   neutral
4950  4950   neutral
4951  4951   neutral
4952  4952  positive

[4953 rows x 2 columns]


In [25]:
# Concatenate all the csv files 1 2 3 4 5
pd.concat([pd.read_csv('/Users/home/Desktop/sentiment/body/1.csv',header=None),
           pd.read_csv('/Users/home/Desktop/sentiment/body/2.csv',header=None),
           pd.read_csv('/Users/home/Desktop/sentiment/body/3.csv',header=None),
           pd.read_csv('/Users/home/Desktop/sentiment/body/4.csv',header=None),
           pd.read_csv('/Users/home/Desktop/sentiment/body/5.csv',header=None)]).to_csv('/Users/home/Desktop/sentiment/body/predictions-body.csv', index=False)

In [27]:
all_prediction = pd.read_csv('/Users/home/Desktop/sentiment/body/predictions-body.csv')
print(len(all_prediction))
print(len(issue_db))
assert len(all_prediction) == len(issue_db)
# delete the 1st column
all_prediction = all_prediction.drop(columns=['0'])
print(all_prediction)
# append it to the issue_db
issue_db['Sentiment_Body'] = all_prediction
print(issue_db.tail())

24953
24953
              1
0      negative
1       neutral
2      positive
3       neutral
4      negative
...         ...
24948   neutral
24949   neutral
24950   neutral
24951   neutral
24952  positive

[24953 rows x 1 columns]
                                                   Title            Name_Repo  \
24948  Cool work. Do you plan to support large games,...  deepmind/open_spiel   
24949                    Infinite loop configuring CMake  deepmind/open_spiel   
24950  add_subdirectory given source "abseil-cpp" whi...  deepmind/open_spiel   
24951                     Failed to invoke `nproc` alias  deepmind/open_spiel   
24952                              Public states support  deepmind/open_spiel   

        State Assignees  Proposed_By Closed_By         Date_Created  \
24948  closed       NaN  lihuiknight   lanctot  2019-08-28 06:23:02   
24949  closed       NaN    bradhowes   lanctot  2019-08-27 16:53:47   
24950  closed       NaN    bradhowes   lanctot  2019-08-27 15:52:01   

In [28]:
# Output the issue_db to a csv file
issue_db = pd.read_csv('issueDB.csv')
issue_db['Sentiment_Body'] = all_prediction
issue_db.to_csv('issueDB.csv', index=False)

In [4]:
# get the max string length of title.txt
with open('/Users/home/Desktop/sentiment/title/title0-5000.csv', 'r') as f:
    print(max(len(line) for line in f))

287


In [19]:
# cut every line of dataframe to 300 characters if it is longer than 300
limit = 300
body_dbs = [pd.read_csv('/Users/home/Documents/PyCharm/Senti4SD/ClassificationTask/body0-.csv', header=None, lineterminator='\n'),
            pd.read_csv('/Users/home/Documents/PyCharm/Senti4SD/ClassificationTask/body5000-.csv', header=None, lineterminator='\n'),
            pd.read_csv('/Users/home/Documents/PyCharm/Senti4SD/ClassificationTask/body10000-.csv', header=None, lineterminator='\n'),
            pd.read_csv('/Users/home/Documents/PyCharm/Senti4SD/ClassificationTask/body15000-.csv', header=None, lineterminator='\n'),
            pd.read_csv('/Users/home/Documents/PyCharm/Senti4SD/ClassificationTask/body20000-.csv', header=None, lineterminator='\n')]
for i in range(len(body_dbs)):
    for j in range(len(body_dbs[i])):
        if len(body_dbs[i][0][j]) > limit:
            body_dbs[i][0][j] = body_dbs[i][0][j][:limit]
            body_dbs[i][0][j] = clean_text(body_dbs[i][0][j])
    body_dbs[i].to_csv(f'/Users/home/Documents/PyCharm/Senti4SD/ClassificationTask/body{i*5000}-{(i+1)*5000}REVISED.csv', header=False, index=False)

In [14]:
test = pd.read_csv('/Users/home/Documents/PyCharm/Senti4SD/ClassificationTask/body0-.csv', header=None)
print(test.tail())

                                                      0
4995  Thanks for your great job!  When reading your ...
4996  Thank you for your great work! Can you tell me...
4997  `OSError: Unable to open file (unable to open ...
4998                           貌似你的代码里面没有任何与NTU120有关的内容
4999  Hello，how to implement skeleton detection of p...
